In [ ]:
import vtracer

input_path = "images/hello.png"
output_path = "outputs/hello.vtracer.svg"


# Single-color example. Good for line art, and much faster than full color:
# vtracer.convert_image_to_svg_py(input_path, output_path, colormode='binary')

vtracer.convert_image_to_svg_py(
    input_path,
    output_path,
    colormode='binary',
    hierarchical='cutout',
    mode='polygon',            # use 'spline' if your art is very curvy
    filter_speckle=30,
    corner_threshold=200,
    length_threshold=40,
    splice_threshold=160,       # ignored in polygon mode, used in spline
    path_precision=2
)                           